In [1]:
# for observability and monitoring
from langsmith import traceable

# to load the model
from langchain_community.llms import Ollama

# to design the prompt template
from langchain_core.prompts import ChatPromptTemplate

# to load the document
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, DirectoryLoader

# to convert document contents into embeddings
from langchain_community.embeddings import OllamaEmbeddings

# document content splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter




In [2]:
import os

In [3]:
# load enviroment variales
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# load a sample document from local file system
loader = DirectoryLoader('/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/',
                         show_progress = True,
                         sample_size = 1,
                         use_multithreading = True)
docs = loader.load()

  0%|          | 0/1 [00:00<?, ?it/s]/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:09<00:00,  9.79s/it]


In [5]:
# visualize the contents of our document
docs[0:5]

[Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy (abridged version).pdf'}, page_content="Executive Summary The advent of Artificial Intelligence (AI) has precipitated a paradigmatic shift in global innovation and transformation, driving cutting-edge advancements across nations and industries. The ramifications of its impact on the global ecosystem and specifically on the developmental trajectory of Nigeria and other African nations are profound and far-reaching. Nigeria, as the continent's most populous country, is singularly positioned to harness AI's transformative capacity to address pressing socio-economic problems, accelerate economic growth, and pivot the nation into a new epoch of technological empowerment, which unlocks unprecedented opportunities for sustainable development, economic prosperity, and human flourishing. Nigeria’s National AI Strategy (NAIS) sets a path to achieve th

In [7]:
# create embedding object to be used in converting our document contents to vector embeddings
embedding_obj = OllamaEmbeddings(model="llama3")

In [8]:
# performing the text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100
)

splitted_doc = text_splitter.split_documents(docs)

In [10]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [11]:
# retrieve api key and cluster url for our qdrant vector database
api_key  = os.getenv("QDRANT_API_KEY")
url = os.getenv("QDRANT_CLOUD_CLUSTER_URL")

#setup qdrant client and collection objects

client = QdrantClient(
    url=url,
    prefer_grpc=True,
    api_key=api_key,
)

client.create_collection(
    collection_name="my_documents",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

True

In [12]:
# creating the vector store object
vector_store = QdrantVectorStore(
    client = client,
    collection_name = "my_documents",
    embedding=embedding_obj,
)

In [13]:
# upload our document to the vectorstore
vector_store.add_documents(splitted_doc)

['f49b215f5ca343fbaa64637a3410a46d',
 'ad623773dd604ba1912d1c4930eb3b09',
 '4b3918af3d17465682b59b650608ac9a',
 '5810235f50134ed4a0f7769e6b315485',
 '3e0bb7cf271d445e9e06129c380341fc',
 'bc6c50039de342698b83ff006a8f5c54',
 'a01a7b6ef4f54b6d82e071664a2be07e',
 '7e2ecb45042f4a0eb664008a884a647f',
 '6abcc17558e84420a5ca8dce9806f662',
 '963ee48f89e8479d92fe2ee2342f7f1d',
 '361dc59539aa495bbb1e4a02bcb63d2b',
 'b90595316ae5464b9a58d49bcacb6b69',
 '2e2c940fb31e45d082033b2e92a8d63a',
 '54c8427b558c462fbd083f3c2678c363',
 '9402cda231da411fb015d22678b76f99',
 '829c829d9616457b834b5105d5c75f04',
 'e223f8012f9c4a6187f0953496b37c98',
 '71561522537647a6b74511fa039d6434',
 '65998eb774ca47fe8a41ac892a0fcea8',
 'd905b7d82dd5467ba0003e19aec5deb4',
 '41d9db8bbe6e4d3389d07aa359b2dce4',
 '774b2af8ecfd48c3bd792ca48c2d80df']

In [14]:
# we write a query which essentially is a question directed at the vetore store while utlizing its similarity search to retrieve response
query = 'summarize the document key points'
response = vector_store.similarity_search_with_score(query)

In [15]:
response

[(Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy (abridged version).pdf', '_id': 'd905b7d8-2dd5-467b-a000-3e19aec5deb4', '_collection_name': 'my_documents'}, page_content='Reduced consumer spending power\n\nAs inflation increases, the purchasing power of Nigerian citizens shrinks. This can lead to decreased demand for AI-powered products and\n\nOpportunity\n\nKnowledge sharing\n\nFunding and investment\n\nThreats\n\nResistance to AI due to feared Job loss/Bias\n\nBrain drain and capital flight\n\nservices, hindering the potential market growth needed to support a thriving AI industry. Consumers may prioritise necessities over adopting new technologies.\n\nRationale'),
  0.5951718091964722),
 (Document(metadata={'source': '/Users/eugene/Personal_Projects/Real_ML_Project/langchain_agent_app/files/National Artificial Intelligence Strategy (abridged version).pdf', '_id': '41d9db8b-be6e-4d33-8

In [16]:
answer, score = response[0]
print(f"Answer: {answer.page_content}")
print(f"Relevance Score: {score}")

Answer: Reduced consumer spending power

As inflation increases, the purchasing power of Nigerian citizens shrinks. This can lead to decreased demand for AI-powered products and

Opportunity

Knowledge sharing

Funding and investment

Threats

Resistance to AI due to feared Job loss/Bias

Brain drain and capital flight

services, hindering the potential market growth needed to support a thriving AI industry. Consumers may prioritise necessities over adopting new technologies.

Rationale
Relevance Score: 0.5951718091964722


In [17]:
# converting the qrant vectorstore to a retriever object
retriever = vector_store.as_retriever()

In [18]:
# Defining the two tools for our agent - a retriever and a search 
# a retriver will let us easily answer questions about the documents
# a search will let us find the most relevant document information for a given question
from langchain.tools.retriever import create_retriever_tool
from langchain.tools.tavily_search import TavilySearchResults

retriever_tool = create_retriever_tool(
    retriever,
    name="retriever",
    description="Searches and returns documents based on their content and metadata.",
)

# setting up the search tool
search = TavilySearchResults()

# creating the final tool to used by our agent
tool = [retriever_tool, search]

In [23]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain import hub
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [29]:
# creating a dummy prompt
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
]
)

llm =  ChatOllama(model="llama3", 
              temperature=0.8,
              num_predict = 256, )

agent = create_openai_functions_agent(
    llm=llm,
    prompt=prompt,
    tools=tool,
)

agent_executor = AgentExecutor(
                    agent=agent,
                    tools = tool,
                    verbose=True
                )

In [25]:
agent_executor.invoke({"input": 'summarize the document key points'})



> Entering new AgentExecutor chain...
As the Executive Secretary to the Minister of Information Technology, I've reviewed the latest document from the Ministry's policy department. The key points are as follows:

**Document:** "National Digital Inclusion Strategy 2023-2025"

**Key Points:**

1. **Objective:** To bridge the digital divide and ensure universal access to information technology across the nation.
2. **Target Audience:** Focus on marginalized communities, including rural populations, persons with disabilities, and low-income households.
3. **Strategies:**
	* Establish a network of community-based digital centers for training and internet access.
	* Develop inclusive digital content and applications that cater to diverse user needs.
	* Implement affordable internet plans and promote digital literacy programs.
4. **Digital Infrastructure:**
	* Upgrade existing national backbone infrastructure to support high-speed connectivity.
	* Expand fiber-optic networks to underserved 

{'input': 'summarize the document key points',
 'output': 'As the Executive Secretary to the Minister of Information Technology, I\'ve reviewed the latest document from the Ministry\'s policy department. The key points are as follows:\n\n**Document:** "National Digital Inclusion Strategy 2023-2025"\n\n**Key Points:**\n\n1. **Objective:** To bridge the digital divide and ensure universal access to information technology across the nation.\n2. **Target Audience:** Focus on marginalized communities, including rural populations, persons with disabilities, and low-income households.\n3. **Strategies:**\n\t* Establish a network of community-based digital centers for training and internet access.\n\t* Develop inclusive digital content and applications that cater to diverse user needs.\n\t* Implement affordable internet plans and promote digital literacy programs.\n4. **Digital Infrastructure:**\n\t* Upgrade existing national backbone infrastructure to support high-speed connectivity.\n\t* Exp